In [4]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

Text for the predictor is in the <code>document</code> variable.

In [11]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")
#predictorOutput = predictor.predict(document="joe is cool. he is my friend.")
predictorOutput = predictor.predict(document="Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers.")

In [23]:
def generateClusters(predictorOutput):
    #function borrowed from: https://github.com/ananyagup/AllenNLP-Coreference-Resolution-in-Python-Readable-clusters
    clusters = predictorOutput['clusters']
    document = predictorOutput['document']
    n = 0
    doc = {}
    for obj in document:
        doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.
        n += 1

    clus_all = []
    cluster = []
    print(doc)
    for i in range(0, len(clusters)):
        one_cl = clusters[i]
        for count in range(0, len(one_cl)):
            obj = one_cl[count]
            for num in range((obj[0]), (obj[1]+1)):
                for n in doc:
                    if num == n:
                        cluster.append(doc[n])
        clus_all.append(cluster)
        cluster = [] 

    print(clus_all) #shows all coreferences
    return clus_all

In [24]:
# generateReferences takes in a context (1+ previous sentences)
# and a current sentence and replaces all broad references with  
# coreferences and the broad reference itself
# TODO: extend list of broad references
broad_refs = ["he", "she", "this", "that", "them", "it"]
def generateReferences(context, curr_sentence):
    perms = []
    clus_all = generateClusters(predictorOutput)
    # need to add something that just grabs the subsentence out of the clusters
    for cluster in clus_all:
        for word in cluster:
                if word in broad_refs:
                    print("word: ", word)
                    results = [curr_sentence.replace(word, newWord) for newWord in cluster if newWord not in broad_refs]
                    print(results)
                    
                    
# example: Paul is cool. He is cool. 
generateReferences("Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers.", "with whom he shared an enthusiasm for computers.")

    

{0: 'Paul', 1: 'Allen', 2: 'was', 3: 'born', 4: 'on', 5: 'January', 6: '21', 7: ',', 8: '1953', 9: ',', 10: 'in', 11: 'Seattle', 12: ',', 13: 'Washington', 14: ',', 15: 'to', 16: 'Kenneth', 17: 'Sam', 18: 'Allen', 19: 'and', 20: 'Edna', 21: 'Faye', 22: 'Allen', 23: '.', 24: 'Allen', 25: 'attended', 26: 'Lakeside', 27: 'School', 28: ',', 29: 'a', 30: 'private', 31: 'school', 32: 'in', 33: 'Seattle', 34: ',', 35: 'where', 36: 'he', 37: 'befriended', 38: 'Bill', 39: 'Gates', 40: ',', 41: 'two', 42: 'years', 43: 'younger', 44: ',', 45: 'with', 46: 'whom', 47: 'he', 48: 'shared', 49: 'an', 50: 'enthusiasm', 51: 'for', 52: 'computers', 53: '.'}
[['Paul', 'Allen', 'Allen', 'he', 'he'], ['Seattle', ',', 'Washington', 'Seattle']]
word:  he
['with whom Paul shared an enthusiasm for computers.', 'with whom Allen shared an enthusiasm for computers.', 'with whom Allen shared an enthusiasm for computers.']
word:  he
['with whom Paul shared an enthusiasm for computers.', 'with whom Allen shared an en